![image](https://cdn.discordapp.com/attachments/996200880351215636/1065002848355631165/New_Atlantis.png) 

---
# Introduction
---

## Initialization Code 

In [16]:
#Code to make directories
#Run this code once you make a new template to automatically generate the accompanying directories
#this will also make empty requirements files and utility nb to be filled in
import os 

if not os.path.exists('./src'):
  os.makedirs('./src')
if not os.path.exists('./Data'):
  os.makedirs('./Data')
if not os.path.exists('./Data/Input_Data'):
  os.makedirs('./Data/Input_Data')
if not os.path.exists('./Data/Output_Data'):
  os.makedirs('./Data/Output_Data')
if not os.path.exists('./Data/Reference_Data'):
  os.makedirs('./Data/Refernce_Data')
if not os.path.exists('./src/requirements.txt'):
  with open('./src/requirements.txt', 'w') as f:
    pass
if not os.path.exists('./src/NB_Utility.ipynb'):
  with open('./src/NB_utility.ipynb', 'w') as nb:
    pass

## Tool Purpose  
---

[antiSMASH](https://github.com/antismash/antismash), is a tool is dedicated to the annotation and analysis of secondary metabolite Biosynthetic Gene Clusters (BGCs) in in archaea, bacteria, and fungi.  
In this NB we will be running this tool to identify BGCs in metagenome assembled genomes (MAGs).

## Input Data 
---

The data required to run this NB are the MAGs (fasta format) and their associated reads alignment mapping files ([BAM format](https://samtools.github.io/hts-specs/SAMv1.pdf)).  
Both the MAGs and mapping files are generated previously utilizing the [VEBA tool](https://github.com/jolespin/veba).  
For demonstration purposes, here we will be analyzing 38 metagenomics samples of the [SOLA dataset](https://pubmed.ncbi.nlm.nih.gov/29925880/), which is a time series dataset spanning three years (from 2012 to 2015) obtained from a coastal northwestern Mediterranean site.

## Output Data 
---

Desctiption of what data is required to be input into the tool.
- Qualitative (Functional Capaticy, Proteomic Assembly etc)
- File types and formatting (.fasta, Blast6, csv/dataframe with schema etc)

The output of this NB consists of antiSMASH output, that is the gbk files created for each BGC annotated.

---
# Environment
---

In [29]:
%load_ext rpy2.ipython
%env REPO=/home/epereira/workspace/dev/new_atlantis/repos/bgc_annot
%env seqtk=/nfs/bin/seqtk/seqtk

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
env: REPO=/home/epereira/workspace/dev/new_atlantis/repos/bgc_annot
env: seqtk=/nfs/bin/seqtk/seqtk


---
## Dependencies (point to requirements.txt and textual list) 
---

Python requirements can be found at `./src/requirements.txt`

[Docker](https://www.docker.com/)  
[tidyverse R package](https://www.tidyverse.org/)  
[compositions R package](https://cran.r-project.org/web/packages/compositions/)  
[biopython Python module](https://biopython.org/)  
[pysam Python module](https://pysam.readthedocs.io/en/stable/)  

In [6]:
!pip3 install import-ipynb -q
!pip3 install pysam -q 
!pip3 install biopython -q
!Rscript -e 'install.packages("tidyverse")' &> /dev/null
!Rscript -e 'install.packages("compositions")' &> /dev/null

---
## Import Statements (code)(import ipynb)
---

In [3]:
import import_ipynb 

In [10]:
%%R
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(compositions))

---
# Parameters
---

Define folder for input data:

In [58]:
%env INPUT_DIR=./Data/Input_Data/sola/
%env OUTPUT_DIR=./Data/Output_Data/

env: INPUT_DIR=./Data/Input_Data/sola/
env: OUTPUT_DIR=./Data/Output_Data/


`--cpu`: Define number of CPUs to run antiSMASH.

In [31]:
%env CPU=40

env: CPU=40


 `--genefinding-tool`: Specify the gene finding tool; here we are using `prodigal-m`, that is Prodigal in Metagenomic mode.

In [32]:
%env GENE_FINDING_TOOL=prodigal-m

env: GENE_FINDING_TOOL=prodigal-m


`--taxon`: Specify the taxonomic classification of input sequences (either bacteria or fungi); here we are using `bacteria`.

In [33]:
%env TAXON=bacteria

env: TAXON=bacteria


 `--minlength`: Define the minimum length of sequences to be analyzed; here we are using `5000` bp.

In [35]:
%env MINLENGTH=5000

env: MINLENGTH=5000


`--allow-long-headers`: Prevents long headers from being renamed (this is a flag, no values needed).

## Input and output data directories 
---

These variables are strings that contain the path to the input and output data directories.

In [2]:
input_dir = 'path/to/data'
output_dir = 'path/to/data'

This cell sets an environment variable to be used later. This is a useful way to set parameters for shell scripts or bash used later in the notebook.

In [26]:
import os 

os.environ['MESSAGE'] = 'Hello Worlds'

---
# Data Precleaning (if required) 
---

This section aims to demonstrate any modifications done to the data from the orginal source, to the format it the tool needs for processing. Follwoing the instructions in this section, a user should be able to replicate the steps required to take the original data set (or any similar data set) and perform requistite transformation required for the tool. If there is any metadata, augmentations to refernce data bases or similar required to run your tool, all those steps should also be included here. 
This section should follow these steps:
- Load data into notebook (pd.read_csv, bash csv, etc)(if required)
- Validate the loaded data is the correct format (for user replication) 
- Perform any other necessary validations (depending on protocol/tool)
- Alternate markdown cell explaining manipulation with code cell executing the manipulation.
- Mark each step heading with ### to insert into tabel fo contents 

If any of these steps are not required for your specific tool, do omit them.

---
# Execution of Tool 
---

This section aims to demonstrate how to execute the tool and performs a sample run on test data. This portion of the notebook may be fairly code intensive and is the most important part of the notebook. To improve readibility and clarity, most of the more verbose code sections should be written as fucntions in python, as shell scripts stored in the src directory, or whatever language necessary for the tool you are using.

If the step you hope to perform involves more than a couple lines of code, please see the function definition format in the src/NB_utility.ipynb and wrap you rcode in a function using that format. If you prefer to use bash scripts, wrtie your commands into a shell file, and then execute them in this portion of the maind deck. once your code is wrapped as function in the utility NB, you can inport and run it using the format below.
Each step should be separated by a markdown heading with a brief explanation followed by the necessary code.
Use the parameter variables definied earlier in the notebook as arguments for functions written here. If there is an input that is not already included in the parameters section, include it there.

First we run antiSMASH in all 38 samples of the SOLA dataset.

In [ ]:
%%bash

SCAFFOLDS=$(ls "${INPUT_DIR}"/ERR*/output/scaffolds.fasta)
for SCAFFOLD in (echo "${SCAFFOLD}" | sed "s/.*\/output.*/\1/");
  OUTPUT_DIR="{SAMPLE_NAME}";
  ./src/run_antismash.sh "${SCAFFOLD}" "${OUTPUT_DIR}/antismash" \
  --cpus "${CPU}" \
  --genefinding-tool "${GENE_FINDING_TOOL}" \
  --taxon "${TAXON}" \
  --allow-long-headers \
  --minlength "${MINLENGHT}";
done 2> /dev/null


Secondly, we can run the script `get_cov.py` to compute the coverage of the identified BGCs.  
Before running this tool, we have to concatenate all BGC gbk files from each metagenomic sample produced by antiSMASH.

In [47]:
%%bash

ls -d "${OUTPUT_DIR}/antismash/"ERR* | \
while read LINE; do
  SAMPLE=$(basename "${LINE}")
  cat "${LINE}"/scaffolds/"${SAMPLE}"*.region*.gbk > "${OUTPUT_DIR}/antismash/${SAMPLE}.gbk"
done

Now we can compute the coverage. Note that for this task we need the associated BAM of the assembled data.  

In [60]:
%%bash

ls "${OUTPUT_DIR}/antismash/"*.gbk | \
while read LINE; do
  SAMPLE=$(basename "${LINE}" .gbk)
   ./src/get_cov.py \
  --input_gbk "${OUTPUT_DIR}/antismash/${SAMPLE}.gbk" \
  --input_bam "${INPUT_DIR}/${SAMPLE}/output/mapped.sorted.bam" \
  --sample_name "${SAMPLE}" \
  --output_tsv "${OUTPUT_DIR}/antismash/${SAMPLE}.tsv"
done

cat "${OUTPUT_DIR}/antismash/"*.tsv  > "${OUTPUT_DIR}/bgc_abund.tsv"

---
# Data Post Processing (if required) 
---

## Write to output directory
---
If the tool does not do it automatically, use this cell to write the output data to the output directory defined in the parameter section.

This section aims to contain all the code necessary to perform the data cleaning, formatting or analysis that would be performed on the output of this tool. Use the same formatting as previously mentioned in the execution section of the notebook:
- Offload long code sections to the src/Utility_NB and import the code 
- Add validation to catch errors in and irregularities in the data 
- Alternate code and markdown cells 
- Include a markdown header for each step using ### to add it to the table of contents
- Display data and tranformations were necessary. 

---
# Visualization 
---

If there is a visualization you would like to include here, generate it here.
Phrase the code used to generate the visualization as a function in the format mentioned in the execution section of this notebook.
Place the function is the utility NB such that it can be reused to generate new visualizations on future data. 
If the vizualization has additional options and parameters, there is no need to add them to the parameters section, and those parameters can be included into a miniature parameter section  in this section.

---
# Conclusion
---
Include any final parting thoughts in this section.
This section may also incude:
- Common mistakes and fixes. 
- Debugging tips.
- Contact for the author.
- Any other information you would like to include